**DSCI 100: GROUP PROJECT PROPOSAL**

**I. Introduction**

Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal

Clearly state the question you will try to answer with your project

Identify and describe the dataset that will be used to answer the question


**II. Preliminary exploratory data analysis**

Demonstrate that the dataset can be read from the web into R 

Clean and wrangle your data into a tidy format

Using only training data, summarize the data in at least one table (this is exploratory data analysis). 

An example of a useful table could be one that reports the number of observations in each class, the means of the predictor
variables you plan to use in your analysis and how many rows have missing data. 

Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do
(this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.


In [1]:
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
credit_data<-read_csv('Dataset/default_of_credit_card_clients.csv', skip=1)
credit_data

Rows: 30000 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (25): ID, LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_0, PAY_2, PAY_3,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,⋯,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,20000,2,2,1,24,2,2,-1,-1,⋯,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,⋯,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,⋯,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
29998,30000,1,2,2,37,4,3,2,-1,⋯,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,80000,1,3,1,41,1,-1,0,0,⋯,52774,11855,48944,85900,3409,1178,1926,52964,1804,1
30000,50000,1,2,1,46,0,0,0,0,⋯,36535,32428,15313,2078,1800,1430,1000,1000,1000,1


In [3]:
tidy_credit_data <- credit_data |>
    pivot_longer(cols = PAY_0:PAY_6, names_to = "PAST_MONTH_REPAYMENT", values_to = "REPAYMENT_STATUS") |>
    pivot_longer(cols = BILL_AMT1:BILL_AMT6, names_to = "BILL_NUMBER", values_to = "BILL_AMOUNT") |>
    pivot_longer(cols = PAY_AMT1:PAY_AMT6, names_to = "PAYMENT NUMBER", values_to = "AMOUNT_PAID") 
    
colnames(tidy_credit_data) <- make.names(colnames(tidy_credit_data))

tidy_credit_data

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,default.payment.next.month,PAST_MONTH_REPAYMENT,REPAYMENT_STATUS,BILL_NUMBER,BILL_AMOUNT,PAYMENT.NUMBER,AMOUNT_PAID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
1,20000,2,2,1,24,1,PAY_0,2,BILL_AMT1,3913,PAY_AMT1,0
1,20000,2,2,1,24,1,PAY_0,2,BILL_AMT1,3913,PAY_AMT2,689
1,20000,2,2,1,24,1,PAY_0,2,BILL_AMT1,3913,PAY_AMT3,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
30000,50000,1,2,1,46,1,PAY_6,0,BILL_AMT6,15313,PAY_AMT4,1000
30000,50000,1,2,1,46,1,PAY_6,0,BILL_AMT6,15313,PAY_AMT5,1000
30000,50000,1,2,1,46,1,PAY_6,0,BILL_AMT6,15313,PAY_AMT6,1000


In [4]:
summary_table <- tidy_credit_data |>
    group_by(default.payment.next.month) |>
    summarize(number_observations_in_each_class = n(),
             mean_repayment_status = mean(REPAYMENT_STATUS, na.rm = TRUE),
             mean_bill_amount = mean(BILL_AMOUNT, na.rm = TRUE),
             mean_amount_paid = mean(AMOUNT_PAID, na.rm = TRUE),
             mean_age = mean(AGE, na.rm = TRUE),
             mean_limit_balance = mean(LIMIT_BAL, na.rm = TRUE))

summary_table
                    

default.payment.next.month,number_observations_in_each_class,mean_repayment_status,mean_bill_amount,mean_amount_paid,mean_age,mean_limit_balance
<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,5046624,-0.3300234,45404.82,5828.237,35.41727,178099.7
1,1433376,0.3371760,43470.49,3328.216,35.72574,130109.7


In [ ]:
#distribution of amount_paid

mean_amount_paid_plot <- tidy_credit_data |>
    ggplot(aes(x = AMOUNT_PAID)) +
    geom_bar()

mean_amount_paid_plot

**III. Method**

Explain how you will conduct either your data analysis and which variables/columns you will use. 
Note - you do not need to use all variables/columns that exist in the raw data set. 
In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?

Describe at least one way that you will visualize the results


**VI. Expected outcomes and significance**




What do you expect to find?

What impact could such findings have?

What future questions could this lead to?
